# GWAS causal SNPs identification analysis

This is a tutorial to show how to interprete GWAS analysis using BRICK.

## load packages and data

In [1]:
import BRICK

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['pdf.fonttype'] = 42


url = "neo4j://10.224.28.66:7687"
auth = ("neo4j", "bmVvNGpwYXNzd29yZA==")  

BRICK.config(url=url, auth=auth)
BRICK.config_llm(modeltype='ChatOpenAI', 
                 api_key="sk-kpsteSkpDGl1xBmDEcC7D51b968e43499092826f17286b55",  
                 base_url='http://10.224.28.80:3000/v1', 
                 llm_params={'model_name': 'qwen-max'})

Graph database has been configured and initialized successfully.
LLM has been configured and initialized successfully.


## extract mutation position 

In [2]:
exp_df = pd.read_csv('./data/filtered_mutation.csv',index_col=0)
exp_df.head()

,CHROM,GENPOS,ID,ALLELE0,ALLELE1,INFO,N,TEST,BETA,SE,CHISQ,LOG10P,EXTRA,MaC,P,mut_pos
1962961,10,92590924,.,G,A,1,24412,ADD,-0.130752,0.023870,30.0058,7.36577,NaN,15798,4.307550e-08,NC_000010.11:g.92590924=
1966217,10,92675916,.,G,A,1,25679,ADD,-0.131653,0.021865,36.2539,8.76142,NaN,18761,1.732130e-09,NC_000010.11:g.92675916=
1966472,10,92685036,.,C,T,1,25759,ADD,-0.134525,0.022788,34.8503,8.44847,NaN,17818,3.560660e-09,NC_000010.11:g.92685036=
1966862,10,92700436,.,GT,GTT,1,25441,ADD,-0.124597,0.021669,33.0633,8.04960,NaN,21009,8.920720e-09,NC_000010.11:g.92700436=
1966875,10,92700893,.,T,C,1,25759,ADD,-0.138311,0.022092,39.1975,9.41678,NaN,18984,3.830190e-10,NC_000010.11:g.92700893=


In [4]:
mutation_chrom_pos = list(exp_df['mut_pos'])

## query mutation id

The mutation id in Knowledge Graph is rs-id in dbSNP, the very first step is to align the dataset to rs-id. Here we use CHROM position to query mutation id.

In [5]:
cypher = f"""MATCH (n:Mutation) 
             WHERE ANY(entity IN SPLIT(n.synonym, '|') WHERE entity IN $source_entity_set) RETURN n"""

mut_ids = BRICK.qr.query_cypher(cypher, parameters={'source_entity_set':mutation_chrom_pos})


In [10]:
mut_id_list = list(mut_ids['n.name'])
len(mut_id_list)

94

In [22]:
"rs10811662" in mut_id_list

True

Here we already figure out this dataset is related to "Diabetes Mellitus, Type 2". We are going to find causal SNPs related to T2D.

In [7]:
target_disease = "Diabetes Mellitus, Type 2"

# Find the SNPs directly related to target_disease

In [36]:
query_df = BRICK.qr.query_relation(source_entity_set=mut_id_list, source_entity_type='Mutation',target_entity_set=[target_disease], target_entity_type='Disease|Phenotype')
query_df.head(1)

,path.0.def,path.0.id,path.0.name,path.0.synonym,path.0.type,path.1,path.1.condition,path.1.info_source,path.1.info_source_length,path.1.original_relation,path.1.relation,path.2.def,path.2.id,path.2.name,path.2.synonym,path.2.type
0,"single nucleotide variant in CDKAL1, related to Type II diabetes mellitus",CLINVAR:16026,rs7756992,7756992|NM_017774.3(CDKAL1):c.371+30101A>G|NM_017774.3:c.371+30101A>G|NC_000006.11:g.20679709=|NC_000006.11:g.20679709A>G|NC_000006.12:g.20679478=|NC_000006.12:g.20679478A>G|rs7756992|7756992,Mutation,causally_related_to,[NCBI:54901],[CLINVAR],1,[causally_related_to],causally_related_to,A subclass of DIABETES MELLITUS that is not INSULIN-responsive or dependent (NIDDM). It is characterized initially by INSULIN RESISTANCE and HYPERINSULINEMIA; and eventually by GLUCOSE INTOLERANCE; HYPERGLYCEMIA; and overt diabetes. Type II diabetes mellitus is no longer considered a disease exclusively found in adults. Patients seldom develop KETOSIS but often exhibit OBESITY.\n,MESH:D003924,"Diabetes Mellitus, Type 2",Undef,Disease


## Figure out SNPs indirectly related to phenotype

Firstly, find the SNPs that located gene related to T2D

In [37]:
from pyensembl import EnsemblRelease

def pyensembl_genes_in_region(chrom, start, end, release=104):
    """
    chrom: 染色体号字符串，如 '7'，如果需要带 'chr' 的形式，可转换为不带 'chr'
    start, end: 区间坐标
    release: Ensembl Release 版本号，注意与大多数人类基因组版本对应关系（如 GRCh38 常用 104/105 等），请根据你的数据选择
    返回：列表，每个元素是字典，包含 gene_id, gene_name, start, end, strand
    """
    ensembl = EnsemblRelease(release)
    genes = ensembl.genes_in_region(chrom, start, end)
    result = []
    for g in genes:
        result.append({
            'gene_id': g.gene_id,
            'gene_name': g.gene_name,
            'start': g.start,
            'end': g.end,
            'strand': g.strand,
            # 可能没有直接属性，请根据实际对象属性检查
            'gene_biotype': getattr(g, 'gene_biotype', None)
        })
    return result

In [38]:
ensembl = EnsemblRelease(104)

In [39]:
tmp = []
for CHR, POS in zip(exp_df['CHROM'], exp_df['GENPOS']):
    gid = ensembl.gene_ids_at_locus(CHR, POS-50, POS+50)
    if len(gid) > 0:
        gname = ensembl.gene_name_of_gene_id(gid[0])
    else:
        gname = ''
    tmp.append(gname)

exp_df['genename'] = tmp    

In [40]:
query_gene_name = [x for x in exp_df['genename'].unique() if x != ''] 

In [41]:
query_gene_name

['KIF11',
 'TCF7L2',
 'KCNQ1',
 'MTNR1B',
 'CCND2',
 'ZNF536',
 'ADCY5',
 'CDKAL1',
 'JAZF1']

In [42]:
path_df = BRICK.qr.query_relation(source_entity_set=query_gene_name, 
                                  source_entity_type='Gene', 
                                  target_entity_type='Disease|Phenotype', 
                                  target_entity_set=[ 'Diabetes Mellitus, Type 2'], 
                                  multi_hop=1
                                 )

In [43]:
path_df.head()

path.0.def  \
0                                                 adenylate cyclase 5<loc>:3q21.1<xref>ENSEMBL:ENSG00000173175|OMIM:600293|HGNC:236</xref>   
1                                          JAZF zinc finger 1<loc>:7p15.2-p15.1<xref>HGNC:28917|ENSEMBL:ENSG00000153814|OMIM:606246</xref>   
2  KCNQ1 potassium voltage-gated channel subfamily Q member 1<loc>:11p15.5-p15.4<xref>OMIM:607542|HGNC:6294|ENSEMBL:ENSG00000053918</xref>   
3                                   KIF11 kinesin family member 11<loc>:10q23.33<xref>ENSEMBL:ENSG00000138160|HGNC:6388|OMIM:148760</xref>   
4                                      MTNR1B melatonin receptor 1B<loc>:11q14.3<xref>ENSEMBL:ENSG00000134640|OMIM:600804|HGNC:7464</xref>   

     path.0.id path.0.name  \
0     NCBI:111       ADCY5   
1  NCBI:221895       JAZF1   
2    NCBI:3784       KCNQ1   
3    NCBI:3832       KIF11   
4    NCBI:4544      MTNR1B   

                                                                                                                                                  path.0.synonym  \
0                                                                                                                                                 AC5|DSKOD|FDFM   
1                                                                                                                                                   TIP27|ZNF802   
2  ATFB1|ATFB3|JLNS1|KCNA8|KCNA9|KVLQT1|Kv1.9|Kv7.1|LQT|LQT1|Potassium voltage-gated channel subfamily KQT member 1|RWS|SQT2|Voltage-gated potassium channel|WRS   
3                                                                                                              EG5|HKSP|KNSL1|Kinesin-like protein 1|MCLMR|TRIP5   
4                                                                                              FGQTL2|MEL-1B-R|MT2|Melatonin receptor|Melatonin receptor type 1B   

  path.0.type        path.1  \
0        Gene  co_mentioned   
1        Gene  co_mentioned   
2        Gene  co_mentioned   
3        Gene  co_mentioned   
4        Gene  co_mentioned   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    path.1.condition  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    [Undef, Undef, Undef, Undef, Undef, Undef, Undef, Undef, Undef]   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                  [Undef, Undef, Undef, Undef, Undef, Undef, Undef, Undef, Undef,

,CHROM,GENPOS,ID,ALLELE0,ALLELE1,INFO,N,TEST,BETA,SE,CHISQ,LOG10P,EXTRA,MaC,P,mut_pos,genename
1962961,10,92590924,.,G,A,1,24412,ADD,-0.130752,0.023870,30.0058,7.36577,NaN,15798,4.307550e-08,NC_000010.11:g.92590924=,KIF11
2569658,10,112969723,.,T,G,1,25760,ADD,0.157863,0.026252,36.1617,8.74088,NaN,10625,1.816020e-09,NC_000010.11:g.112969723=,TCF7L2
2569743,10,112973697,.,C,T,1,25757,ADD,0.159670,0.025070,40.5634,9.72048,NaN,12297,1.903360e-10,NC_000010.11:g.112973697=,TCF7L2
2569755,10,112974337,.,A,G,1,25760,ADD,0.178802,0.025234,50.2082,11.85930,NaN,12103,1.382610e-12,NC_000010.11:g.112974337=,TCF7L2
2569774,10,112974984,.,CTTTTTTTTTT,CTTTTTTTTTTT,1,25718,ADD,0.141308,0.022102,40.8756,9.78986,NaN,18893,1.622330e-10,NC_000010.11:g.112974984=,TCF7L2
2569814,10,112976855,.,T,C,1,25759,ADD,0.191009,0.025380,56.6391,13.28100,NaN,11888,5.236000e-14,NC_000010.11:g.112976855=,TCF7L2
2569865,10,112978633,.,ATTTTTTTTT,ATTTTTTTT,1,25148,ADD,0.188480,0.025471,54.7588,12.86560,NaN,11944,1.362700e-13,NC_000010.11:g.112978633=,TCF7L2
2569904,10,112979765,.,T,TA,1,23531,ADD,0.185964,0.026259,50.1521,11.84690,NaN,11890,1.422660e-12,NC_000010.11:g.112979765=,TCF7L2
2569939,10,112980858,.,C,T,1,25759,ADD,0.121683,0.021577,31.8024,7.76782,NaN,20407,1.706790e-08,NC_000010.11:g.112980858=,TCF7L2
2570102,10,112985860,.,TTGTGTGTGTGTGTGTG,TTGTGTGTGTGTGTG,1,25493,ADD,0.189159,0.024947,57.4942,13.46980,NaN,11724,3.390000e-14,NC_000010.11:g.112985860=,TCF7L2


In [50]:
res = BRICK.inp.interpret_results('This is an anlysis result of GWAS data of type 2 diabete(t2d) samples, list all reported t2d related SNPs and generate hypothesis based on snps indirectly related to t2d, try to remain the original information as possible, do not be lazy',
                            SNPs_directly_related_to_T2D = query_df[['path.0.name', 'path.1', 'path.2.name']], 
                            SNPs_locate_gene_related_to_T2D = exp_df.loc[exp_df['genename'].isin(query_gene_name)])

print(res)

INFO:httpx:HTTP Request: POST http://10.224.28.80:3000/v1/chat/completions "HTTP/1.1 200 OK"


### 1. Reasoning the Analysis Result

#### Key Evidence Chain and Statistical Significance
1. **Directly Related SNPs:**
   - The analysis identifies 38 SNPs that are causally related to Type 2 Diabetes (T2D). These SNPs, such as `rs7756992`, `rs849135`, and `rs10830963`, have been previously reported in GWAS studies.
   - The statistical significance of these associations is indicated by the `LOG10P` values, which range from 7.36 to 44.05. A `LOG10P` value of 7.36 corresponds to a p-value of \(10^{-7.36} \approx 4.3 \times 10^{-8}\), which is highly significant.

2. **Indirectly Related SNPs:**
   - The analysis also identifies 224 SNPs that are located within genes related to T2D. These SNPs, such as those in `TCF7L2`, `KCNQ1`, and `ADCY5`, show strong associations with T2D.
   - The `LOG10P` values for these SNPs range from 7.36 to 44.05, indicating high statistical significance. For example, `rs112969723` in `TCF7L2` has a `LOG10P` value of 9.74, corresponding to a p-value of \(10^